<a href="https://colab.research.google.com/github/mansivyas26/perfusion_dataPipeline/blob/main/variables_diff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import numpy as np
import random
import ast
import warnings
from pandas_gbq import to_gbq
from google.cloud import storage
from google.cloud import bigquery
from io import StringIO


import pickle

warnings.filterwarnings('ignore')

In [5]:
!pip install upgrade pandas

ERROR: Could not find a version that satisfies the requirement upgrade (from versions: none)
ERROR: No matching distribution found for upgrade


In [ ]:
!pip install --upgrade google-cloud-storage


In [123]:
from google.cloud import storage
import pandas as pd
import pickle

def download_pickle_from_gcs(bucket_name):
    try:
        # Initialize a client
        key_path = '/content/drive/MyDrive/Pefusion_DataPipeline/platform-developement-dev-env-7a2997352c88.json'  # Replace with your service account key path
        client = storage.Client.from_service_account_json(key_path)

        # Get the bucket
        bucket = client.get_bucket(bucket_name)

        # Specify the folder name
        folder_name = 'Steve/'

        # List all blobs (files) in the specified folder
        blobs = bucket.list_blobs(prefix=folder_name)

        # Iterate over each blob in the specified folder
        for blob in blobs:
            # Check if the blob is a file, starts with 'BEXT...', and ends with 'all_data.pkl'
            if blob.name.startswith(folder_name) and blob.name.startswith(folder_name + 'BEXT') and blob.name.endswith('all_data.pkl'):
                # Download the file
                content = blob.download_as_string()

                # Load the pickle object from the downloaded content
                pickle_object = pickle.loads(content)

                # Convert the pickle object to a DataFrame
                df = pd.DataFrame(pickle_object)

                # Print the DataFrame
                print("DataFrame:")
                #print(df)

                # Print success message
                print("Load success!!!")

                return df

        # If no matching file is found
        print("No file matching the criteria found in the specified folder.")
        return None

    except Exception as e:
        print(f"Error: {e}")
        return None


In [136]:
bucket_name = "mark2_runs_bex_ds_dev"
file_name = "all_data.pkl"
perfusion_individual_pickle_data = download_pickle_from_gcs(bucket_name)


DataFrame:
Load success!!!


In [125]:

from google.cloud import storage
import pandas as pd
from io import StringIO

def download_csv_from_gcs(bucket_name, file_name):
    try:
        # Initialize a client
        key_path = '/content/drive/MyDrive/Pefusion_DataPipeline/platform-developement-dev-env-7a2997352c88.json'  # Replace with your service account key path
        client = storage.Client.from_service_account_json(key_path)

        # Get the bucket
        bucket = client.get_bucket(bucket_name)

        # Specify the CSV file path in the bucket
        file_path = f"{file_name}.csv"  # Adjust the file extension if needed

        # Get the blob (file) from the bucket
        blob = bucket.blob(file_path)

        # Download the file content as bytes
        content_bytes = blob.download_as_bytes()

        # Decode the bytes content to string
        content_str = content_bytes.decode('utf-8')

        # Convert the CSV content to a DataFrame
        df = pd.read_csv(StringIO(content_str))

        # Print the DataFrame
        print("DataFrame:")
        #print(df)

        # Print success message
        print("Load success!!!")

        return df

    except Exception as e:
        print(f"Error: {e}")
        return None


In [279]:
bucket_name = "perufsion_data_variable_dictionary"
file_name = "variable_dict_2802 - test_262.02 - test"
variable_dict = download_csv_from_gcs(bucket_name,file_name)
variable_dict['possible_variables'] = variable_dict['possible_variables'].apply(ast.literal_eval)

DataFrame:
Load success!!!


In [293]:
variable_dict.loc[variable_dict['standard_variable'] == 'syringe_pump_volume_setpoint', 'variable_status'] = 'EXISTING'

In [127]:

from google.cloud import bigquery
import pandas as pd

def read_bq_table_to_df(project_id, dataset_id, table_id,key_path):
    try:
        # Initialize a BigQuery client
        client = bigquery.Client.from_service_account_json(key_path, project=project_id)
        # Specify the reference to the table
        table_ref = client.dataset(dataset_id).table(table_id)

        # Fetch the table data
        table = client.get_table(table_ref)

        # Execute a query to get the data
        query_job = client.query(f"SELECT * FROM `{project_id}.{dataset_id}.{table_id}` LIMIT 100")

        # Fetch the results as a Pandas DataFrame
        df = query_job.to_dataframe()

        # Print the DataFrame
        print("DataFrame:")
       # print(df)

        # Print success message
        print("Load success!!!")

        return df

    except Exception as e:
        print(f"Error: {e}")
        return None


In [283]:
project_id = 'platform-developement-dev-env'
dataset_id = 'combine_runs'
table_id = 'combined_runs'
key_path = '/content/drive/MyDrive/Pefusion_DataPipeline/platform-developement-dev-env-7a2997352c88.json'

schema = read_bq_table_to_df(project_id, dataset_id, table_id,key_path)


DataFrame:
Load success!!!


In [284]:
perfusion_data_subset = perfusion_individual_pickle_data.head(3)

In [285]:
#check for perfusion ID
import pandas as pd

def check_columns(df):
    required_columns = ['perfusion_id', 'perfusion_date', 'device_ID']

    # Check if required columns are present
    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        return f"Missing columns: {', '.join(missing_columns)}"

    # Check if required columns have non-empty, non-None, non-NaN values
    missing_values_indices = df[df[required_columns].isnull().any(axis=1)].index.tolist()

    if missing_values_indices:
        return f"Missing values in rows: {', '.join(map(str, missing_values_indices))}"

    return "All conditions satisfied"

# Example usage:


result = check_columns(perfusion_data_subset)
print(result)


Missing columns: perfusion_id, perfusion_date, device_ID


In [286]:
#check scehma with variable dictionary and renames the column if needed to standard_variable
def check_schema(schema,variable_dict):
  columns = schema.columns.tolist()
  for column in columns:

        for idx, row in variable_dict.iterrows():
            if column in row['possible_variables'] and column != row['standard_variable']:
                print("old names",column,row['possible_variables'])
                schema.rename(columns={column: row['standard_variable']}, inplace=True)
                print("columns that were renamed",column)


In [287]:
#function call
check_schema(schema,variable_dict)

old names glucose_control_mode ['glucose_control_mode_auto_flag', 'glucose_control_mode', 'auto_glucose', 'Auto Glucose', 'Auto_Glucose']
columns that were renamed glucose_control_mode
old names syringe_pump_manual_setrate ['glucose_pump_rate_automatic_setpoint', 'glucose_syringe_pump_hour_setrate', 'syringe_pump_manual_setrate', 'glucose_pump_rate_manual_setpoint', 'glucose_pump_rate_setpoint', 'glucose_pump_rate', 'syringe_pump_rate', 'glucose_pump_rate_setpoint']
columns that were renamed syringe_pump_manual_setrate
old names time ['time', 'time(in hrs)', 'time_hrs']
columns that were renamed time
old names probe_ph_mv_measured ['probe_ph_mv_measured', 'pH_mV', 'mV', 'mettler_ph_mv']
columns that were renamed probe_ph_mv_measured
old names glucose_pump_rate_setpoint ['glucose_pump_rate_automatic_setpoint', 'glucose_syringe_pump_hour_setrate', 'syringe_pump_manual_setrate', 'glucose_pump_rate_manual_setpoint', 'glucose_pump_rate_setpoint', 'glucose_pump_rate', 'syringe_pump_rate', 'g

In [58]:

def check_columns(df):
  val = 'None'
  if val in df.columns:
      # Drop the column named 'None'
      df = df.drop(columns=[val])


In [59]:
check_columns(perfusion_data_subset)

In [60]:
check_columns(schema)

In [288]:
temporary_none = ['dialysis_compensate_scale_weight','dialysis_flux_valve_position','viscometer_temperature','dialysis_post_pressure','dialysis_compensate_pump_rate','dialysis_compensate_scale_flux','dialysis_pre_pressure','dialysis_pre_pressure_setpoint','dialysis_pressure_valve_position','dialysis_waste_scale_flux','dialysis_waste_scale_weight']

In [289]:
for val in temporary_none:

        # Add a new column to the pkl DataFrame with the value initialized as None
  perfusion_data_subset[val] = "None"

In [294]:
#compare .pkl file , variable dicitionary and Schema

def check_runs_with_variable_dict_new(pkl_df, variable_dict, schema_subset):

    removed_columns = []  # List to keep track of REMOVED columns
    list_of_files_ignored_in_pkl = ['timestamp','glucose_syringe_pump_buzzer']
    list_of_files_ignored_in_schema = ['resistance_qc_solenoid_flag','perfusion_date','perfusion_id','control_time','glucose_syringe_pump_dispensed_control', 'glucose_syringe_pump_buzzer_control']

   # temporary_none = ['dialysis_compensate_scale_weight','dialysis_flux_valve_position','viscometer_temperature','dialysis_post_pressure','dialysis_compensate_pump_rate','dialysis_compensate_scale_flux','dialysis_pre_pressure','dialysis_pre_pressure_setpoint','dialysis_pressure_valve_position','dialysis_waste_scale_flux','dialysis_waste_scale_weight']
    #list of variables to be added in schema
    for idx, row in variable_dict.iterrows():
            if row['standard_variable'] in list_of_files_ignored_in_schema:
                  pkl_df[row['standard_variable']] = "to be added"
    columns = pkl_df.columns.tolist()
    for column in columns:
        match_found = False
        for idx, row in variable_dict.iterrows():
           if column in row['possible_variables']:
                match_found = True
                if row['variable_status'] == "EXISTING" and column != row['standard_variable']:
                    pkl_df.rename(columns={column: row['standard_variable']}, inplace=True)
                    if column in schema_subset.columns:
                        schema_subset.rename(columns={column: row['standard_variable']}, inplace=True)
                elif row['variable_status'] == "NEW":

                    #add a column with name row['standard_variable'] and  value = None
                    variable_dict.loc[idx,'variable_status'] = "EXISTING"
                  #rename to standard variable
                    if row['standard_variable'] not in schema_subset.columns:
                        schema_subset[row['standard_variable']] = None

                elif row['variable_status'] == "REMOVED":
                   # print(f"Removed column identified: {column}")
                    removed_columns.append(column)
                    for c in removed_columns:
                      print(removed_columns)
                      pkl_df[c] = None




    cols_to_drop = list(set(pkl_df.columns).intersection(set(list_of_files_ignored_in_pkl)))
    print(cols_to_drop)
    pkl_df = pkl_df.drop(cols_to_drop,inplace=True,axis=1)



    print("printed")









In [303]:
def check_columns_match(df1, df2):
    columns_df1 = set(df1.columns)
    columns_df2 = set(df2.columns)

    if columns_df1 != columns_df2:
        columns_only_in_df1 = columns_df1 - columns_df2
        columns_only_in_df2 = columns_df2 - columns_df1

        print("Columns only in .pkl file:", columns_only_in_df1)
        print("Columns only in schema:", columns_only_in_df2)
        print(len(columns_only_in_df2),"only_schema")
        print(len(columns_only_in_df1),"only_pkl")

        return False
    else:
        print(len(df1.columns),"pkl")
        print(len(df2.columns),"schema")
        print("Columns match between the two DataFrames.")
        return True

def append_df_row_wise(pkl_df, schema_df):
    #pkl_df = pkl_df.drop(columns=['None'],inplace=True)

    # Check if columns match
    if not check_columns_match(pkl_df, schema_df):
      return None
    else:
      schema_df = schema_df[sorted(schema_df.columns)]
      pkl_df = pkl_df[sorted(pkl_df.columns)]



      #add rows only if perfusion_id looks like BEXT
      combined_df = schema_df.append(pkl_df,ignore_index=True)

      #combined_df.to_csv("/content/drive/MyDrive/Pefusion_DataPipeline/combined.csv")
     # print(combined_df)
      return combined_df



In [ ]:
#create individual table for dryrun - filter perfusion dataset convert bq table

In [ ]:
#create indivudual table for human runs - filter perfusion dataset convert bq table

In [304]:
combined_df = append_df_row_wise(perfusion_data_subset,schema)

133 pkl
133 schema
Columns match between the two DataFrames.


In [305]:
combined_df['B_dialysis_loop_on_off']

KeyError: 'B_dialysis_loop_on_off'

In [295]:
print(check_runs_with_variable_dict_new(perfusion_data_subset,variable_dict,schema))

['control_time']
['control_time', 'glucose_syringe_pump_buzzer_control']
['control_time', 'glucose_syringe_pump_buzzer_control']
[]
printed
None


In [297]:
list1 = ['glucose_syringe_pump_volume_setpoint', 'syringe_pump_volume_setpoint', 'glucose_pump_rate_automatic_setpoint']
list2 = schema.columns.to_list()
list3 = perfusion_data_subset.columns.to_list()

In [298]:
def find_matching_values(list1, list2):
    return list(set(list1) & set(list2))

In [300]:
len(find_matching_values(list2,list3))
#find_matching_values(glucose_syringe_pump_volume_setpoint,glucose_syringe_pump_hour_setrate)

132

In [65]:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound


def create_table_and_load_df(project_id, dataset_id, table_id, df, key_path):
    # Initialize BigQuery client with service account key
    client = bigquery.Client.from_service_account_json(key_path, project=project_id)
    dataset_ref = client.dataset(dataset_id)

    # Create dataset if it doesn't exist
    dataset = bigquery.Dataset(dataset_ref)
    if not client.get_dataset(dataset_ref, retry=None):
        dataset = client.create_dataset(dataset)

    table_ref = dataset_ref.table(table_id)

    # Define the schema based on the DataFrame data types
    schema = []
    for column, dtype in df.dtypes.items():
        if dtype == 'int64':
            schema.append(bigquery.SchemaField(column, 'INTEGER'))
        elif dtype == 'float64':
            schema.append(bigquery.SchemaField(column, 'FLOAT64'))
        elif dtype == 'object':
            schema.append(bigquery.SchemaField(column, 'STRING'))
        # Add more conditions for other data types as needed

    # Create the table with the specified schema
    table = bigquery.Table(table_ref, schema=schema)
    table = client.create_table(table)

    # Load the DataFrame into the table
    job_config = bigquery.LoadJobConfig(schema=schema)
    client.load_table_from_dataframe(df, table_ref, job_config=job_config).result()

    print(f'Data loaded into BigQuery table {project_id}.{dataset_id}.{table_id}.')


In [68]:
project_id = 'platform-developement-dev-env'
dataset_id = 'dataConsistency_test'
table_id = 'BEXT84_updated_test'
key_path = "/content/drive/MyDrive/Pefusion_DataPipeline/platform-developement-dev-env-7a2997352c88.json"

# Call the function to create the table (if it doesn't exist) and load the DataFrame
create_table_and_load_df(project_id, dataset_id, table_id, perfusion_data_subset_copy,key_path)

Data loaded into BigQuery table platform-developement-dev-env.dataConsistency_test.BEXT84_updated_test.


In [206]:
from collections import Counter

my_list = schema.columns.to_list()  # Example list

# Count occurrences of each element
counts = Counter(my_list)

# Find duplicate values
duplicates = [item for item, count in counts.items() if count > 1]

print("Duplicate values:", duplicates)

Duplicate values: ['glucose_syringe_pump_hour_setrate']


In [205]:
schema['glucose_control_mode_auto_flag']

0     None
1     None
2     None
3     None
4     None
      ... 
95    None
96    None
97    None
98    None
99    None
Name: glucose_control_mode_auto_flag, Length: 100, dtype: object

In [101]:
perfusion_data_subset_columns = perfusion_data_subset.columns.to_list()
check_list_probe_o2_temp = ['probe_o2_temperature', 'Mettler_O2_Temp', 'pO2 temperature', 'Mettler O2 Temp']
print(perfusion_data_subset_columns)
val = "probe_o2_temperature"
if val in perfusion_data_subset_columns:
  print("val foud ",val)
else:
  print("not found")


['time', 'pulse_generator_base_lower_bound', 'raman_uncertainty_95_measured', 'brain_flow_measured', 'mettler_o2_percent_air', 'main_pump_rate_setpoint', 'glucose_syringe_diameter', 'glucose_pump_rate_automatic_setpoint', 'mettler_o2_g_l', 'update_controls_var_flag', 'glucose_syringe_pump_volume_dispensed_measured', 'dialysis_compensate_pump_setrate', 'dialysis_pre_pressure_setpoint', 'gas_mixer_flow_o2_measured', 'syringe_pump_volume_setpoint', 'heat_exchanger_temperature_setpoint', 'efficiency_gas_mixer_n2', 'glucose_syringe_pump_buzzer', 'brain_pressure_target', 'pulse_generator_amp_pressure_range_setpoint', 'brain_flow_range_calculated', 'brain_pressure_range_target', 'mettler_ph_mv', 'gas_mixer_co2_setpoint', 'post_qpv_pressure', 'glucose_control_mode_auto_flag', 'mettler_co2_dli', 'gas_mixer_o2_maximum', 'gas_mixer_flow_measured', 'gas_mixer_percent_o2_measured', 'perfusion_time', 'efficiency_main_pump', 'main_pump_oscillation_amplitude', 'raman_glucose_mg_dl_measured', 'brain_sc

In [98]:
def common_elements(list1, list2):
    common_values = [value for value in list1 if value in list2]
    return common_values

In [320]:
schema[['perfusion_id','air_supply_pressure']]

,perfusion_id,air_supply_pressure
0,BEXI1,None
1,BEXI1,None
2,BEXI1,None
3,BEXI1,None
4,BEXI1,None
...,...,...
95,BEXI1,None
96,BEXI1,None
97,BEXI1,None
98,BEXI1,None


In [319]:
combined_df[['perfusion_id','air_supply_pressure']]

,perfusion_id,air_supply_pressure
0,BEXI1,None
1,BEXI1,None
2,BEXI1,None
3,BEXI1,None
4,BEXI1,None
...,...,...
98,BEXI1,None
99,BEXI1,None
100,to be added,NaN
101,to be added,323.0
